<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Quartile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/ckdresearch2023/Assign/Assign 6: MissingValue/MissForest.xlsx')
df = df.iloc[:, 1:]
df

,PatientUID,PatientVisitUID,Slopes,InPatient,OutPatient,Emergency,time_t1,eGFR_t1,VisitLocation,StartDTTM,...,L14-WBC,L15-Platelet,L19-Blood-sugar,L20-HbA1c,L21-Cholesterol,L22-Triglyceride,L23-LDL-Cholesterol,L24-HDL-Cholesterol,L28-UrineProtein-Cr-ratio,LabLocation
0,2299,3386255,-7.158596,0,1,0,2017-10-03 09:19:54.710,63.60,เวชปฏิบัติทั่วไป (OPD)-D03,2559-08-15 08:13:26.043,...,5.8998,256.67,91.00,5.559,215.0,81.00,123.00,99.00,65.810,1
1,2299,4546471,5.550691,0,1,0,2018-12-12 09:18:49.913,70.20,เวชปฏิบัติทั่วไป (OPD)-D03,2560-10-03 07:37:22.777,...,5.6377,260.39,86.00,5.489,219.0,64.00,146.00,73.00,105.881,1
2,2299,5812902,-6.063710,0,1,0,2019-12-19 09:51:24.427,64.02,เวชปฏิบัติทั่วไป (OPD)-D03,2561-12-12 07:58:57.190,...,9.4100,313.00,100.00,5.300,193.0,62.00,129.00,65.00,51.688,1
3,2299,6947220,-12.812245,0,1,0,2021-03-04 10:01:42.113,48.54,เวชปฏิบัติทั่วไป (OPD)-D03,2562-12-19 08:19:38.647,...,4.9215,206.32,89.00,5.414,205.0,69.00,139.00,76.00,20.270,1
4,2299,8186767,28.287500,0,1,0,2022-04-21 08:45:54.580,80.47,เวชปฏิบัติทั่วไป (OPD)-D03,2564-03-04 07:55:40.703,...,5.4200,434.00,88.00,5.500,198.0,74.00,129.00,60.00,47.001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52615,3319961,9270330,-4.256753,0,1,0,2023-04-04 08:28:58.833,33.68,บูรณาการนอกเวลา (คลินิก)-OD191,2565-03-14 14:31:31.910,...,10.3200,306.00,103.00,5.900,165.0,311.00,84.00,43.00,131.300,1
52616,3319971,9270331,-0.692078,0,1,0,2023-04-04 08:49:20.047,26.47,บูรณาการนอกเวลา (คลินิก)-OD191,2565-03-14 14:31:53.720,...,9.5600,361.00,99.00,7.100,154.0,196.00,83.00,42.00,325.800,1
52617,3320048,9242721,7.217875,0,0,1,2023-04-10 08:36:15.217,70.36,OPD AE-DV191,2565-03-05 12:36:33.263,...,4.6100,131.00,123.34,6.511,112.0,96.00,64.00,33.00,63.789,1
52618,3320050,9257281,-32.794557,0,1,0,2023-04-10 08:58:57.267,30.69,หน่วยคลินิกเฉพาะทางศูนย์บริการทางการแพทย์ชั้นเลิศ,2565-03-10 07:42:44.243,...,9.4400,305.00,131.06,6.871,206.4,152.73,149.15,44.13,181.380,1


In [3]:
df = df[['PatientUID','PatientVisitUID','Slopes']]
df

,PatientUID,PatientVisitUID,Slopes
0,2299,3386255,-7.158596
1,2299,4546471,5.550691
2,2299,5812902,-6.063710
3,2299,6947220,-12.812245
4,2299,8186767,28.287500
...,...,...,...
52615,3319961,9270330,-4.256753
52616,3319971,9270331,-0.692078
52617,3320048,9242721,7.217875
52618,3320050,9257281,-32.794557


In [4]:
df.isnull().any()

PatientUID         False
PatientVisitUID    False
Slopes             False
dtype: bool

In [5]:
data = pd.DataFrame(df)

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Define the bins
bins = [-80, -5, -0.75, 107]

# Categorize the data
data['Slopes_Category'] = pd.cut(data['Slopes'], bins=bins, labels=['Low', 'Moderate', 'High'])

# Number of splits
n_splits = 5

# Initialize KFold
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Store the results
results = []

# Perform the data splitting
for fold, (train_index, test_index) in enumerate(kf.split(data)):
    train_data, test_data = data.iloc[train_index], data.iloc[test_index]

    # Calculate value counts for each category in the fold
    value_counts = train_data['Slopes_Category'].value_counts()
    results.append((fold, len(train_data), value_counts))

    # Display the data in each fold
    #print(f"Fold {fold + 1} Train Data:")
    #print(train_data)
    #print(f"Fold {fold + 1} Test Data:")
    #print(test_data)
    #print("---------------")

# Display the fold number, number of rows, and value counts for each fold
for i, result in enumerate(results):
    fold_num, fold_size, value_counts = result
    print(f"Fold {fold_num + 1}:")
    print(f"Number of Rows: {fold_size}")
    print(f"Value Counts:\n{value_counts}")
    print("---------------")


Fold 1:
Number of Rows: 42096
Value Counts:
High        18385
Low         13082
Moderate    10629
Name: Slopes_Category, dtype: int64
---------------
Fold 2:
Number of Rows: 42096
Value Counts:
High        18418
Low         13054
Moderate    10624
Name: Slopes_Category, dtype: int64
---------------
Fold 3:
Number of Rows: 42096
Value Counts:
High        18298
Low         13104
Moderate    10694
Name: Slopes_Category, dtype: int64
---------------
Fold 4:
Number of Rows: 42096
Value Counts:
High        18424
Low         13019
Moderate    10653
Name: Slopes_Category, dtype: int64
---------------
Fold 5:
Number of Rows: 42096
Value Counts:
High        18395
Low         13089
Moderate    10612
Name: Slopes_Category, dtype: int64
---------------


In [ ]:
data

,PatientUID,PatientVisitUID,Slopes,Slopes_Category
0,2299,5812902,-7.158596,Low
1,2299,6947220,5.550691,High
2,2299,8186767,-6.063710,Low
3,2299,9385133,-12.812245,Low
4,2304,4916314,28.287500,High
...,...,...,...,...
52614,3319961,10553903,10.568365,High
52615,3319971,10553916,-4.256753,Moderate
52616,3320048,10572126,-0.692078,High
52617,3320050,10572565,7.217875,High


## **check**

In [ ]:
train_data

,PatientUID,PatientVisitUID,Slopes,Slopes_Category
0,2299,5812902,-7.158596,Low
1,2299,6947220,5.550691,High
3,2299,9385133,-12.812245,Low
4,2304,4916314,28.287500,High
5,2304,6395251,-8.126415,Low
...,...,...,...,...
52613,3314119,10421285,-1.342317,Moderate
52614,3319961,10553903,10.568365,High
52615,3319971,10553916,-4.256753,Moderate
52617,3320050,10572565,7.217875,High


In [ ]:
test_data

,PatientUID,PatientVisitUID,Slopes,Slopes_Category
2,2299,8186767,-6.063710,Low
9,2312,7765456,-7.411572,Low
10,2312,8842427,24.938876,High
12,2354,8438646,4.056631,High
16,2362,8169235,7.488793,High
...,...,...,...,...
52593,3294009,10480935,-4.492308,Moderate
52599,3298298,10281892,7.391940,High
52600,3299347,10328531,-6.835095,Low
52604,3301951,10442151,8.212500,High


# **Excel**

In [ ]:
quartile.to_excel("Quartile.xlsx", index=False)